# No es lo mismo que lo mesmo
[__Mariana Esther Martínez-Sánchez__](https://github.com/mar-esther23/)

Un problema común es comparar secuencias de texto que pueden tener pequeñas diferencias, ya sea por la forma de captura o por errores de ortografía. En este taller veremos tres estrategias de limpieza y análisis de datos: 
* __Comparación de strings__: usar [operaciones de strings](https://www.w3schools.com/python/python_ref_string.asp), [expresiones regulares](https://docs.python.org/2/library/re.html) básicas y [unidecode](https://pypi.org/project/Unidecode/) para manejar carácteres especiales
* __Estandarizar con catálogo__: usar [Fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy), una herramienta para determinar cuán similares son dos secuencias de texto usando la [distancia de Levenshtein](https://es.wikipedia.org/wiki/Distancia_de_Levenshtein), para estandarizar datos a un catálogo predefinido
* __Stemming__: un método para reducir una palabra a su raíz o [stem](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html) usando [NLTK](https://www.nltk.org/api/nltk.stem.html) y [Snowball](https://www.nltk.org/_modules/nltk/stem/snowball.html).

__Requisitos__: Computadora con [anaconda](https://www.anaconda.com/distribution/) instalado e instalar los paquetes de [requierements.txt](https://stackoverflow.com/questions/51042589/conda-version-pip-install-r-requirements-txt-target-lib/51043636).

### Contenido
1. [Comparación de strings](#strings)
    * Operaciones de strings
    * Ejemplo
2. [Estandarizar con catálogo](#fuzzywuzzy)
    * Fuzzywuzzy
    * Ejemplo
3. [Stemming](#stemming)
    * NLTK y snowball
    * Ejemplo

### Caso de estudio: Guerra Sucia

La Guerra sucia son las medidas de represión militar y política encaminadas a disolver a los movimientos de oposición política y armada contra el Estado mexicano durante las décadas de 1960 a 1980. La guerra sucia dejó un número aún desconocido de muertos y desaparecidos, aunque se estima que fueron alrededor de 1500. El análisis de estos datos se ve dificultado por varias razones: ocultamiento del estado, regiones de difícil acceso, contradicciones entre fuentes, etc.

En este caso usaremos una lista de desaparecidos de la Guerra Sucia. En la primera columna hay una lista de desaparecidos en la base de datos, mientras que en la segunda columna hay otra lista de desaparecidos que queremos integrar. Algunos de estos podrían ser nombres repetidos, escritos de diferente forma o desaparecidos que no están en el catálogo.

In [1]:
import pandas as pd
from IPython.display import display

df = pd.read_csv("DesaparecidosGuerraSucia.csv")
display(df.head())

catalogo = df['Catalogo'].tolist()
datos = df['Faltantes'].dropna().tolist()
print('N catal: ', len(catalogo), '\t\tN datos: ', len(datos))

,Catalogo,Faltantes
0,Abad Torres Meza,Abel Baltazar Ramírez
1,Abel Alfaro Silva,Abelino Llanes Ponciano
2,Abel Almazán Saldaña,Adanabe Solón Guzmán Cruz
3,Abel Balanzar Ramírez,Adelino Francisco Gallango Cruz
4,Abel Estrada Camarillo,Adenauer Solón Guzmán Cruz


N catal:  937 		N datos:  63


Verifiquemos cuantos de los nombres faltantes se encuentran en el catálogo

In [2]:
def contar_match(datos, catalogo):
    """Imprimir el número de elementos que se encuentran en ambas listas"""
    print('Match:', sum([d in catalogo for d in datos]), ' de ', len(datos) )
    
contar_match(datos, catalogo)

Match: 6  de  63


<a id='strings'></a>

## 1. Limpieza básica

Un problema común en la busqueda de personas es estandarizar las diferentes bases de datos para integrar la información de los desaparecidos y tratar de encontrarlos. Sin embargo, muchas veces las diferentes bases de datos tienen errores de captura, faltas de ortografía o usaán distintos formatos.

[Maria Teresa Torres Ramírez](https://biblioteca.archivosdelarepresion.org/s/comverdad/item?Search=&property%5B0%5D%5Bproperty%5D=58&property%5B0%5D%5Btype%5D=eq&property%5B0%5D%5Btext%5D=Mar%C3%ADa%20Teresa%20Torres%20Ram%C3%ADrez%20de%20Mena%20) desaparecio en 1976 en Acapulco Guerrero, donde era estudiante de la Preparatoria 7 de la UAG. En el momento de su desaparición estaba embarazada de tres meses, [no se sabe que fue de ella ni de su hijo](https://www.proceso.com.mx/154026/donde-estan-teresa-y-su-hijo).

### Operaciones de strings

Python tiene una serie de operaciones para procesar strings las cuales pueden ser usadas como ayuda a la limpieza de estos.

Por ejemplo:

In [3]:
text  = "María Teresa\tTorres Ramírez  (de Mena). \n"
print(text)

María Teresa	Torres Ramírez  (de Mena). 



Si hay mas de un espacio o tabulador entre palabras es posible removerlo usando _.join()_ y _.split()_.

In [4]:
' '.join([t for t in text.split()])

'María Teresa Torres Ramírez (de Mena).'

Las computadoras tratan las letras mayúsculas y minúsculas como diferentes carácteres, por lo cual puede ser útil concertir todas al mismo formato usando _.lower(), .upper()_ o _.title()_.

In [5]:
text.lower()

'maría teresa\ttorres ramírez  (de mena). \n'

Otro problema común son los caracterés especiales como puntos, comas y acentos. Estos pueden ser eliminados usando expresiones regulares como _re.sub()_.

Nota: Para quitar números usar: '[^A-Za-z ]+'

In [6]:
import re
re.sub('[^A-Za-z0-9 ]+', '', text)

'Mara TeresaTorres Ramrez  de Mena '

Los acentos son tratados como carácteres especiales, por lo cual muchas veces requieren un tratamiento especial. En ese caso _unidecode()_ permite sustituir los acentos por las vocales correspondientes.

In [7]:
import unidecode
unidecode.unidecode(text)

'Maria Teresa\tTorres Ramirez  (de Mena). \n'

Generalmente la mejor opción es combinar estas operaciones en una función de limpieza de texto.

In [8]:
def clean_string(text):
    """Limpiar un texto para dejar solo letras minúsculas sin acentos o carácteres especiales"""
    text = text.lower()                        #pasar a minusculas
    text = ' '.join([t for t in text.split()]) #simplificar espacios, tabs, etc
    text = unidecode.unidecode(text)           #recuerda cambiar acentos...
    text = re.sub('[^A-Za-z0-9 ]+', '', text)  #antes de quitar caracteres especiales 
    text = ' '.join([t for t in text.split()]) #paranoia
    return text

clean_string(text)

'maria teresa torres ramirez de mena'

### Ejemplo

Usando esta función podemos tratar de mejorar la coincidencia entre nuestros datos y el catálogo.

In [9]:
datos_clean = [clean_string(d) for d in datos]
catalogo_clean = [clean_string(c) for c in catalogo]

contar_match(datos_clean, catalogo_clean)

Match: 9  de  63


<a id='fuzzywuzzy'></a>

## 2. Estandarizar con catálogo

En algunos casos los datos son categóricos, es decir, pertenecen a un número limitado de grupos. Un catálogo permite clasificar y ordenar los datos. Sin embargo, los datos pueden ser capturados con errores, lo cual dificulta a las computadoras trabajarlos como datos categóricos. 

### Fuzzywuzzy

Fuzzywuzzy es una biblioteca que calcula la distancia de Levenshtein, es decir, la cantidad de inserciones, eliminaciones o sustitucioes de caracteres que se necesitan para transformar una string en otro.
Es decir, es una forma de determinar que tanto se parecen dos strings.

In [10]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

texts = ["María Teresa\tTorres Ramírez  (de Mena). \n",
         "Ma. Teresa Torres Ramírez (embarazada de 3 meses) ", 
         "Ma  Teresa Torres Ramires",
         "Torres Ramírez María Teresa "]

La función _fuzz.ratio()_ permite determinar la distancia entre dos strings.

In [11]:
fuzz.ratio("María Teresa Torres Ramírez", 
           "Ma  Teresa Torres Ramires")

85

Esto se puede mejorar combinando varias estrategias, por ejemplo, limpiar los strings y luego compararlos.

In [12]:
fuzz.ratio(clean_string("María Teresa Torres Ramírez"), 
           clean_string("Ma  Teresa Torres Ramires"))

90

Sin embargo _fuzz.ratio()_ tiende a cometer errores cuando el problema es que las palabras cambian de lugar.

In [13]:
fuzz.ratio("María Teresa Torres Ramírez", 
           "Torres Ramírez María Teresa ")

51

En este caso es mejor usar _fuzz.token_sort_ratio()_. Esta función toma cada palabra como un "token" y compara que los "token"s se parezcan, sin importar el orden.

In [14]:
fuzz.token_sort_ratio("María Teresa Torres Ramírez", 
                      "Torres Ramírez María Teresa ")

100

Otra opción es comparar el texto de interes contra un catálogo usando _process.extract()_. Esto regresa el catálogo según su similitud al texto de interes.

In [15]:
process.extract("María Teresa Torres Ramírez", texts)

[('María Teresa\tTorres Ramírez  (de Mena). \n', 95),
 ('Torres Ramírez María Teresa ', 95),
 ('Ma  Teresa Torres Ramires', 88),
 ('Ma. Teresa Torres Ramírez (embarazada de 3 meses) ', 86)]

### Ejemplo

Usando fuzzywuzzy podemos estandarizar los datos a un catálogo ya existente. 

Queremos:
* Para cada nombre en los datos faltantes encontrar el nombre mas similar del catálogo
* Verificar manualmente los casos donde puede haber dudas sin tener que comparar con toda la base de datos
* Crear una diccionario de alias que podamos usar en otros procesos
* Evitar hacer la misma comparación varias veces

In [16]:
def match_datos_con_catálogo(datos, catalogo, thr_aceptacion=90, n_opciones=3):
    """
    Generar un diccionario de mapeo entre un conjunto de datos y un catalogo usando fuzzywuzzy.
    
    Parametros:
    datos: list
        Lista de strings con los datos a estandarizar con el catálogo
    catalogo: list
        Lista de strings con el catalogo al que se quieren mapear los datos
    thr_aceptacion: int, default 90
        Similitud mínima con la mejor opción que se requiere, si no se cumple se muestran las opciones para revisión manual
    n_opciones: int, defaul 3
        Número de opciones a revisar manualmente en caso de que no haya una similitud mínima
   
    Regresa:
        Diccionario con todos los datos y la opción del catálogo para ese dato.
    """
    dic = {}
    # solo hacer cada comparación una vez
    datos = set(datos)
    catalogo = set(catalogo)
    for nombre in datos:
        #comparar nombre con catálogo
        res = process.extract(nombre, catalogo, limit=n_opciones)
        top_nombre, top_score = res[0]
        #guardar mejor opción en diccionario
        dic[nombre] = top_nombre
        #verificar manualmente si la mejor opción esta abajo de thr_aceptacion
        if top_score<thr_aceptacion:
            print("BAJA SIMILITUD {}:\t'{}'\n\t{}".format(top_score, nombre, [r[0] for r in res]))
    return dic

mapeo = match_datos_con_catálogo(datos, catalogo, thr_aceptacion=90, n_opciones=3)

BAJA SIMILITUD 86:	'Adenauer Solón Guzmán Cruz'
	['Sulpicio de Jesús De la Cruz Bautista', 'Angel Cruz Mayo', 'Solón Adanabe Guzmán Cruz']
BAJA SIMILITUD 88:	'Ramona Ríos García'
	['Romana Ríos García', 'Juan Ignacio Mendoza García', 'Alicia De los Ríos Merino']
BAJA SIMILITUD 81:	'Inocencio Bello Ríos'
	['Ausencio Bello Ríos', 'Inocencio Calderón ', 'Bonifacio Bello Malo']
BAJA SIMILITUD 68:	'Cristina Rocha de Herrera'
	['Cristina Rocha Manzanares', 'Juan de Dios Herrera Sánchez', 'Alberto López Herrera']
BAJA SIMILITUD 88:	'Candencio Moreno González'
	['Laurencio Moreno González', 'Angel Moreno Ríos', 'Fernando González ']
BAJA SIMILITUD 84:	'Santana Yáñez Noriega'
	['J. Santana Llanes Noriega', 'Francisco Tabares Noriega', 'Jacinto Noriega Zavala']
BAJA SIMILITUD 88:	'Melitón Arreola González'
	['Melitón Arroyo González', 'Sixto González ', 'Laurencio Moreno González']
BAJA SIMILITUD 87:	'Juan José Barreras Valenzuela'
	['Juan Enrique Barreras Valenzuela', 'José Luis Martínez', 'Jos

La revisión manual nos muestra varios errores, podemos corregir estos errores alterando el diccionario con las opciones que consideramos adecuadas.

In [17]:
mapeo['Adenauer Solón Guzmán Cruz'] = 'Solón Adanabe Guzmán Cruz' #es la tercera opción
mapeo['Humberto Cruz Ávila'] = 'Humberto Cruz Ávila' #no esta en catálogo, nuevo desaparecido?
mapeo['Romana Ríos de Roque'] = 'Romana Ríos García' #encontrada revisando catálogo

Muchas veces es buena idea revisar el diccionario una última vez manualmente. (No se muestra el resultado)

```python
for k,v in mapeo.items()[0:5]:
    print(k, '\t\t',v)
```

Este mapeo es mucho mas exitoso que las anteriores estrategias:

In [18]:
datos_fuzzy = [mapeo[d] for d in datos]

contar_match(datos_fuzzy, catalogo)

Match: 62  de  63


<a id='stemming'></a>

## Stemming

Existen métodos pará análizar datos que son textos, los cuales tienden a ser mas largos y a tener una mayor diversidad de palabras y puntuación. Un método común para análizar textos es tratar de determinar cuales son las palabras mas usadas en un texto para obtener información del tema de este. 
Sin embargo, una dificultad común es que existen palabras con terminaciones diferentes pero que se refieren al mismo concepto, por ejemplo "persona" y "personas". Para mejorar los análisis de textos se reducen estas palabras a su "raíz", quitando los modificadores del final de la palabra. Este proceso se conoce como _stemming_.
Además se quitan palabras muy comunes pero que dan poca información del significado del texto como "de", "las", "eso", "un", "mas".

### NLTK 

NLTK es una bibliteca que contiene múltiples herramientas para estudiar lenguaje natural.


In [19]:
from nltk import word_tokenize, download
from nltk.stem import SnowballStemmer
download('punkt')

[nltk_data] Downloading package punkt to /home/esther/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

La función _.word_tokenize()_ permite separar una oración en una lista de palabras y puntuación. Esta función requiere descargar _punkt_.

In [20]:
text = "Persona (personas), personal, personalidad, personaje."
text = word_tokenize(text)
text

['Persona',
 '(',
 'personas',
 ')',
 ',',
 'personal',
 ',',
 'personalidad',
 ',',
 'personaje',
 '.']

La función _stemmer.stem()_ obtiene la raíz de cada palabra, quitando las términaciones de manera eurística. Cade idioma utiliza diferentes modificadores, por lo que es necesario usar un stemmer especial. En este caso usaremos _SnowballStemmer('spanish')_.

In [21]:
stemmer = SnowballStemmer('spanish')

for t in text:
    if len(t)>1:
        print(t, '\t', stemmer.stem(t))

Persona 	 person
personas 	 person
personal 	 personal
personalidad 	 personal
personaje 	 personaj


### Ejemplo

Una forma de estudiar un texto es saber que palabras y terminos usa comunmente. 

Vamos a definir dos funciones por simplicidad del código. La primera _word_freq()_ sirve para contar cuantas veces aparece cada palabra en un texto. La segunda _stem_text_ obtiene todas las raices del texto.

In [22]:
def word_freq(text, min_len=4):
    """Contar la frequencia de palabras de un texto. Las palabras debén de tener mas de min_len caracteres."""
    text = [i for i in word_tokenize(text) if len(i)>=min_len] #dividir texto
    dic  = {s:text.count(s) for s in set(text)} #contar apariciones de palabras
    dic =  {k: dic[k] for k in sorted(dic, key=dic.get, reverse=True)} #ordenar diccionario
    return dic

def stem_text(text):
    """Obtener las raices de todas las palabras de un texto."""
    text = ' '.join( [stemmer.stem(i) for i in word_tokenize(text)] )
    return text

En este caso usaremos el texto completo [El canto del cisne de la FEMOSPP: La única condena a un perpetrador de la guerra sucia en México](https://adondevanlosdesaparecidos.org/2020/01/27/el-canto-del-cisne-de-la-femospp-la-unica-condena-a-un-perpetrador-de-la-guerra-sucia-en-mexico/).

In [23]:
filename = "Yankelevich2020-FEMOSPP.txt"
with open(filename) as f:
    text = f.read()

Usando estas funciones podemos saber cuales son las palabras mas usadas para darnos una idea rápida del tema del texto.

Nota: este conteo puede ser mejorado usando estrategias de limpieza.

In [24]:
thr_print = 15

text_freq = word_freq( text )

for k,v in text_freq.items():
    if v >= thr_print:
        print(k, '\t', v)

para 	 33
FEMOSPP 	 32
desaparición 	 30
Miguel 	 30
Guzmán 	 28
como 	 19
México 	 19
1977 	 17
sobre 	 16
Estado 	 15
forzada 	 15


Sin embargo, este conteo es muy sensible a la terminación de las palabras, lo cual puede hacer que se pierdan términos importantes. Para mejorar esto usaremos stemming.

In [25]:
stem = stem_text(text)
stem_freq = word_freq(stem)

for k,v in stem_freq.items():
    if v >= thr_print:
        print(k, '\t', v)

desaparicion 	 37
femospp 	 32
miguel 	 32
guzman 	 30
investig 	 28
agent 	 22
mexic 	 21
fiscal 	 20
federal 	 18
estad 	 18
penal 	 18
forz 	 18
1977 	 17
sobr 	 17
person 	 17
este 	 16


Esta es una forma rápida de permitir que una computadora "estudie" el tema de un texto automáticamente, sin necesidad de que sea leido por un ser humano.

In [26]:
print(text[0:1000])

El canto del cisne de la FEMOSPP: La única condena a un perpetrador de la guerra sucia en México

Por Javier Yankelevich*, enero 27, 2020

Los estudios sobre la FEMOSPP han descrito e intentado explicar su fracaso. El más reciente dice “ninguno de los casos llevados ante las cortes penales resultaron en condenas. De hecho, ninguno de ellos alcanzó la etapa de juicio”. Esto es impreciso pues, en septiembre de 2009, Esteban Guzmán Salgado, ex agente de la Dirección Federal de Seguridad (DFS), fue condenado por la desaparición forzada de Miguel Ángel Hernández Valerio, comenzada en 1977 en Mazatlán y continuada hasta la fecha. El juicio que culminó de este modo había arrancado, en 2006, con la consignación de una Averiguación Previa foliada como PGR/FEMOSPP/018/2004. De esto no se enteraron ni los ex integrantes de la FEMOSPP, para ese momento dispersados por otras áreas de la PGR o despedidos.

Los balances sobre la FEMOSPP difícilmente serán desafiados porque un solo responsable de las 

# FIN

Espero que estas estrategias para trabajar con texto les sean útiles.

Muchas gracias!